In [ ]:
import magmaforge
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
comp={
    'BSE': {
        'SiO2': 45.97,
        'TiO2':  0.18,
        'Al2O3': 4.77,
        'Fe2O3': 1e-7,
        'FeO':   8.24,
        'MnO':   0.0,
        'MgO':  36.66,
        'CaO':   3.78,
        'Na2O':  0.35,
        'K2O':   0.04,},
    'BSE_MS95': {
        'SiO2': 45.5,
        'TiO2':  0.00001,
        'Al2O3': 4.50,
        'FeO':   8.15,
        'MnO':   0.0,
        'MgO':  38.3,
        'CaO':   3.58,
        'Na2O':  0.00001,
        'K2O':   0.0,},
}


P = 24e3
T0 = 2200.0
Tfinal = 1600 + 273-100

BSE = comp['BSE_MS95']
dNNO = -5

model_name = 'pMELTS'

In [ ]:
comp_IW0 = magmaforge.System.adjust_init_redox(T0, P, BSE, dNNO, 'NNO', model_name)
Tliq = magmaforge.System.get_liquidus_temp(T0, P, comp_IW0, model_name)
comp_IW = magmaforge.System.adjust_init_redox(Tliq, P, BSE, dNNO, 'NNO', model_name)

print(Tliq)

In [ ]:

comp_non_Fe = comp_IW.drop(labels=['FeO','Fe2O3']) 
comp_non_Fe0 = (comp_IW0*comp_IW['SiO2']/comp_IW0['SiO2']).drop(labels=['FeO','Fe2O3'])
assert np.allclose(comp_non_Fe, comp_non_Fe0), 'Non-Fe oxide components should not change.'

In [ ]:
sys_T = magmaforge.System(comp=comp_IW, T0=Tliq, P=P,
                          melt_frac_cutoff=.15, Tfinal=Tfinal, model_name=model_name)

Sliq = sys_T.total_entropy
print(Sliq)

sys_S = magmaforge.System(comp=comp_IW, T0=Tliq, P=P, 
                          min_potential='H', S0=Sliq, model_name=model_name)

print(sys_S.total_entropy)

print(sys_T.state.conditions)
print(f"Fe2O3 = {sys_T.state.liquid.comp['Fe2O3']}")

print(sys_S.state.conditions)
print(f"Fe2O3 = {sys_S.state.liquid.comp['Fe2O3']}")

In [ ]:
sys_T.crystallize(method='equil', Tstep=5)

In [ ]:
while(sys_S.mass_fraction > 0.15):
    sys_S.cool(dS=1)
    print(sys_S.T)
    if sys_S.T < Tfinal:
        break

In [ ]:
plt.figure()
plt.plot(sys_S.history.get_temps()-273, sys_S.history.get_melt_frac(),'.-', label='S')
plt.plot(sys_T.history.get_temps()-273, sys_T.history.get_melt_frac(),'--', label='T')
plt.xlabel('T')
plt.ylabel('melt frac')
plt.legend()

plt.figure()
plt.plot(sys_S.history.get_temps()-273, sys_S.history.get_total_entropy(),'.-', label='S')
plt.plot(sys_T.history.get_temps()-273, sys_T.history.get_total_entropy(),'--', label='T')
plt.xlabel('T')
plt.ylabel('Stot')

In [ ]:
magmaforge.plot.magma_evolution(sys_T.history, T_lims=[1475, 1700])

In [ ]:
magmaforge.plot.magma_evolution(sys_S.history, T_lims=[1475, 1700])